Vamos analisar várias ações e escolher a melhor (segundo um critério que não é o único, mas serve de ilustração)
 
Falar de ações na bolsa é falar de dois conceitos que sempre andam juntos: Retorno e Risco.

### Retorno
Quando você compra uma ação, você espera que o seu preço suba. Se você compra uma ação por um preço $p_0$ e vende por um preço $p_1$, o **retorno** que você teve com essa ação foi de

$$
\frac{p_1-p_0}{p_0}
$$

Por exemplo, se você compra uma ação por 20 e vende poor 22, você investiu 20  ganhou 2, ou seja, 10%. Usando a fórmula acima, nosso resultado seria

$$
\frac{22-20}{20}=\frac{2}{20}=0.10
$$

ou seja, 10\%.

Vamos fazer uma função que recebe uma lista $p$ (de _preços_) contendo preços de ação ao longo de um período e nos devolva o retorno da ação ao longo desse período.

In [33]:
def retorno(p):
    return((p[-1]-p[0])/p[0])

### Risco

O risco de uma ação é a incerteza sobre o seu retorno.  
Ao comprarmos uma ação, nós gostaríamos que o seu preço subisse. Mas não temos certeza. Ele pode cair. Se subir, ganhamos dinheiro. Se cair, perdemos. Essa incerteza é o risco da ação.  

Como podemos medir o risco de uma ação? Você vai ter que aprender um pouco de Estatística para responder essa questão. Por ora, acredite em mim, e vamos usar a função abaixo. Ela recebe uma lista contendo preços de ação ao longo de um período, e te retorna o risco da ação neste mesmo período. Note que o input dela é igual ao input da função `retorno`.

In [34]:
def risco(p):
    
    '''
    Calcula o risco (volatilidade) de uma ação ao longo de um período
    
    Parametros
    ----------
    p: lista de floats
        Lista contendo os preços da ação ao longo de um período
    
    Retorna
    -------
    float
        Retorno da ação ao longo do período correspondente à lista de preços inputada

    Exemplo
    -------
    >>> p = [1,1.1,1.2,0.9]
    >>> risco(p)
    ... 0.3990001277297724
    '''
    
    import pandas as pd
    import numpy as np
    
    #Calcula retorno das ações
    retornos_diarios = pd.Series(p).pct_change()
    
    #Calcula risco correspondente a 1 dia
    risco_1_dia = retornos_diarios.std()
    
    #Calcula risco correspondente ao período total
    risco_periodo = risco_1_dia * np.sqrt(len(p))
    
    return(risco_periodo)

Você sabia? Os _traders_ profissionais chamam o risco de uma ação de **volatilidade**. Uma ação volátil é uma ação muito arriscada.

### Relação retorno-risco

Alguns investimentos não têm risco.  
É o caso de um Título do Tesouro Nacional, por exemplo.
Comprar um Título do Tesouro Nacional é emprestar dinheiro para o governo.  
O governo _vai_ te pagar. Não há risco dele dar calote ([quer dizer...](https://economia.uol.com.br/noticias/redacao/2019/08/28/relembre-casos-de-paises-que-cairam-na-moratoria.htm) deixa quieto.)

Se eu empreso dinheiro para o governo, o governo me paga a [Taxa Sélic](https://www.bcb.gov.br/controleinflacao/taxaselic). Portanto, eu consigo aplicar meu dinheiro à taxa Sélic sem risco nenhum.  

Aplicar em ações tem risco. Por isso, eu só vou aplicar em ações se for para ganhar _mais_ do que a taxa Selic. O "quanto a mais eu ganho" se chama _prêmio de risco_: 

$$
\text{premio de risco} = \text{retorno da ação} - \text{taxa Selic}
$$

Em outras palavras, o prêmio de risco é "quanto eu ganho com risco" menos "quanto eu ganho sem risco".

A relação entre esse ganho adicional e o risco que eu estou correndo para ter esse ganho se chama **Índice de Sharpe**:

$$
\text{Índice de Sharpe} = \frac{\text{premio de risco}}{\text{risco da ação}}
$$

O indice de Sharpe, portanto, mede a relação entre retorno e risco de uma ação.

* Se o índice de Sharpe é alto, a ação é boa, porque significa que ela me dá muito retorno com pouco risco;
* Se o índice de Sharpe é baixo, a ação é ruim, porque significa que ela me dá pouco retorno com muito risco;


Vamos fazer uma função chamada `Sharpe` para calcular o índice de Sharpe de uma ação ao longo de um período. 
A função deve receber o mesmo input que as funções `retorno` e `risco` anteriormente, ou seja, uma lista $p$ com os preços de uma ação ao longo de um período. Além disso, a função também deve receber um parâmetro correspondente à taxa Selic. Esse parâmetro deve ter como valor _default_ o valor da taxa Sélic de hoje. Você pode consultar o valor da taxa Sélic hoje neste site: [Selic Hoje](https://www.selichoje.com.br/) e atualizar o valor do default, se quiser. 

Dica: O retorno e a volatilidade são dados _sem_ o sinal de porcentagem. Um retorno de 0.10, por exemplo, significa um retorno de 10%. Para manter a coerencia e evitar erros, faça o mesmo com a taxa Sélic: Se a Selic estiver a 2%, por exemplo, o input deverá ser 0.02, e não 2.

In [47]:
def Sharpe(p, selic=0.02): #valor default atualizado dia 7/12/2020
    premio_de_risco = retorno(p) - selic
    indice_de_sharpe = premio_de_risco/risco(p)
    return(indice_de_sharpe)

### Teste com uma ação

Vamos usar as funções que você criou na prática. Mas antes, vamos ver se ela funciona. Para tal, vamos usar uma única ação: Petrobrás.

Na bolsa, cada ação é representada por um código, chamado _ticker_. A Petrobrás é representada pelo ticker **PETR4**.  

Para baixar da internet os preços da ação da Petrobrás, vamos precisar do pacote `investpy`:


In [48]:
#!pip install investpy

In [49]:
import investpy

A função `get_stock_recent_data` busca dados recentes de uma ação, inclusive preço. Vamos entender melhor como ela funciona:

In [50]:
investpy.get_stock_recent_data?

Signature:
investpy.get_stock_recent_data(
    stock,
    country,
    as_json=False,
    order='ascending',
    interval='Daily',
)
Docstring:
This function retrieves recent historical data from the introduced stock from Investing.com. So on, the recent data
of the introduced stock from the specified country will be retrieved and returned as a :obj:`pandas.DataFrame` if
the parameters are valid and the request to Investing.com succeeds. Note that additionally some optional parameters
can be specified: as_json and order, which let the user decide if the data is going to be returned as a
:obj:`json` or not, and if the historical data is going to be ordered ascending or descending (where the index is the 
date), respectively.

Args:
    stock (:obj:`str`): symbol of the stock to retrieve recent historical data from.
    country (:obj:`str`): name of the country from where the stock is.
    as_json (:obj:`bool`, optional):
        to determine the format of the output data, either a :obj:

Vamos tenta reproduzir o exemplo usando a ação da Petrobrás:

In [51]:
petrobras = investpy.get_stock_recent_data('petr4','brazil')
petrobras

,Open,High,Low,Close,Volume,Currency
Date,,,,,,
2020-11-09,21.11,22.69,21.05,21.61,163120000,BRL
2020-11-10,21.89,23.15,21.82,23.08,161144896,BRL
2020-11-11,23.35,23.35,22.41,22.88,91735504,BRL
2020-11-12,22.74,22.81,21.77,21.91,75142496,BRL
2020-11-13,22.03,22.76,21.90,22.63,63995000,BRL
2020-11-16,23.20,23.63,23.16,23.29,96512704,BRL
2020-11-17,23.05,24.14,22.99,23.69,92762800,BRL
2020-11-18,23.97,24.13,23.55,23.55,64340100,BRL
2020-11-19,23.54,23.93,23.38,23.82,63380000,BRL


Quando falamos em "preço" de uma ação, normalmente queremos dizer "preço de fechamento", ou seja, o preço com o qual a ação termina o dia.

Vamos selecionar o preço de fechamtno, apenas:

In [52]:
petrobras = petrobras['Close']
petrobras

Date
2020-11-09    21.61
2020-11-10    23.08
2020-11-11    22.88
2020-11-12    21.91
2020-11-13    22.63
2020-11-16    23.29
2020-11-17    23.69
2020-11-18    23.55
2020-11-19    23.82
2020-11-20    23.65
2020-11-23    25.10
2020-11-24    26.22
2020-11-25    26.25
2020-11-26    25.82
2020-11-27    25.50
2020-11-30    24.90
2020-12-01    25.60
2020-12-02    25.91
2020-12-03    26.64
2020-12-04    27.53
2020-12-07    27.00
Name: Close, dtype: float64

Nós precisamos de 2 passos para chegar ao preço final da Petrobrás:

1. Usar a função `get_stock_historical_data` para buscar os preços da internet
2. Selecionar apenas preço de fechamento

Como nós estaremos fazendo isso várias vezes (e como, conceitualmente, esses dois passos fazem uma coisa só -- buscar o preço da ação), vamos criar uma função que nos permita fazer isso em um passo só:

In [53]:
def busca_preco_fechamento(ticker):
    
    '''
    Busca preço de fechamento de uma ação
    
    Parametros:
    ----------
    ticker: str
        Ticker da ação
    
    Retorna:
    -------
    lista
        Lista com os preços de fechamento de uma ação
        ao longo do último mês.
        
    
    Exemplo
    -------
    >>> busca_preco_fechamento('petr4')
    '''
    
    todos_os_precos = investpy.get_stock_recent_data(ticker, country='Brazil')
    preco_fechamento = todos_os_precos['Close']
    return(preco_fechamento)

In [54]:
###Vamos testar nossa função
busca_preco_fechamento('petr4')

Date
2020-11-09    21.61
2020-11-10    23.08
2020-11-11    22.88
2020-11-12    21.91
2020-11-13    22.63
2020-11-16    23.29
2020-11-17    23.69
2020-11-18    23.55
2020-11-19    23.82
2020-11-20    23.65
2020-11-23    25.10
2020-11-24    26.22
2020-11-25    26.25
2020-11-26    25.82
2020-11-27    25.50
2020-11-30    24.90
2020-12-01    25.60
2020-12-02    25.91
2020-12-03    26.64
2020-12-04    27.53
2020-12-07    27.00
Name: Close, dtype: float64

### Quase lá!

Agora, vamos criar um dicionário vazio chamado `desempenho`.

Em seguida, vamos calcular o retorno, o risco e o índice de Sharpe da Petrobrás, e colocar isso no dicionário. Ao final, o dicionário deverá ter 3 chaves, chamadas retorno, risco e sharpe, com seus respectivos valores. Algo tipo assim:

```python
{'retorno': 0.123,
 'risco': 0.456,
 'sharpe': 0.789}
```
(os meus números podem não bater com os teus. Isso porque estamos rodando em datas diferentes. Eu tô fazendo isso dia 6 de dezembro de 2020. E você?)

In [65]:
desempeno={}
p = busca_preco_fechamento('petr4')
r = retorno(p)
sigma = risco(p)
sh = Sharpe(p)
desempenho = {'retorno': r, 'risco': sigma, 'sharpe': sh}
desempenho

{'retorno': 0.2494215640906988,
 'risco': 0.13380165621431858,
 'sharpe': 1.7146391949231163}

### It's show time, baby!

O código abaixo gera uma lista com 100 das ações mais importantes da bolsa de valores (não se preocupa com o que "mais importante" significa. Relaxa.)

Esse código vai demorar uns 10 segundos para rodar...

In [66]:
lista_de_acoes = list(investpy.get_stocks_overview('brazil').symbol)

Vamos calcular o índice de Sharpe para todas as ações dessa lista, para selecionar qual delas tem o maior índice de Sharpe. Essa é, potencialmente, uma boa ação para se estudar mais a fundo, e ver se vale a pena comprar.  

Partiu calcular o índice de Sharpe de todas as ações da lista!!!

In [71]:
!pip install tqdm # -- coloca uma barra de progresso para avisar quanto tempo falta no loop

ERROR: Invalid requirement: '#'


In [69]:
from tqdm import tqdm

desempenho = {} #Crie um dicionário vazio chamado "desempenho"

sucessos, fracassos = 0, 0
for s in tqdm(lista_de_acoes): # O for loop deve pegar cada ação da lista de ações
    try:
        p = busca_preco_fechamento(s) #o preço de fechamento da ação
        retorno_da_acao = retorno(p) #o retorno da ação. Use a função que você criou!
        risco_da_acao = risco(p) #o risco da ação. Use a função que eu criei!
        indicde_de_sharpe = Sharpe(p) #o índice de sharpe. Use a função que você criou
        desempenho[s] = {'retorno': retorno_da_acao,
                         'risco': risco_da_acao, 
                         'sharpe': indicde_de_sharpe} #Crie um dicionário
                                                               #contendo 3 chaves:
                                                               # retorno, risco e sharpe,
                                                               #contendo os valores que você
                                                               #acabou de calcular
        sucessos += 1
    except:
        fracassos += 1



100%|██████████| 100/100 [00:49<00:00,  2.01it/s]


In [72]:
desempenho

{'ABCB4': {'retorno': 0.12997746055597284,
  'risco': 0.08058867897265994,
  'sharpe': 1.364676303892302},
 'AGRO3': {'retorno': 0.1292281006071119,
  'risco': 0.12024885044610567,
  'sharpe': 0.9083504765483545},
 'RAIL3': {'retorno': -0.022982885085574518,
  'risco': 0.1027477081698674,
  'sharpe': -0.41833424658497653},
 'ALPA3': {'retorno': 0.00946547884187072,
  'risco': 0.07335834105062942,
  'sharpe': -0.14360359036552794},
 'ALPA4': {'retorno': 0.01636363636363643,
  'risco': 0.09221360036875724,
  'sharpe': -0.03943413576546135},
 'ALSO3': {'retorno': 0.035945363048166784,
  'risco': 0.13289937633496066,
  'sharpe': 0.11998072141420711},
 'AMAR3': {'retorno': 0.0,
  'risco': 0.11962709740099273,
  'sharpe': -0.16718620140852827},
 'ABEV3': {'retorno': 0.05274261603375527,
  'risco': 0.12210775018328157,
  'sharpe': 0.2681452732083687},
 'ADHM3': {'retorno': 0.04477611940298523,
  'risco': 0.16566072143793809,
  'sharpe': 0.14955940785436683},
 'ARZZ3': {'retorno': 0.0806774441

As ações estão em ordem de sei-lá-o-quê. Queremos que elas fiquem em ordem decrescente do Índice de Sharpe, com o maior índice aparecendo primeiro.

Podemos fazer isso com o seguinte comando:

In [73]:
desempenho_em_ordem = sorted(desempenho.items(), key = lambda x: x[1]['sharpe'], reverse=True)
desempenho_em_ordem

[('EEEL3',
  {'retorno': 0.16666666666666666,
   'risco': 0.019965281992164446,
   'sharpe': 7.346085405867411}),
 ('TASA4',
  {'retorno': 1.0578313253012044,
   'risco': 0.3334715357107358,
   'sharpe': 3.112203634080042}),
 ('OIBR4',
  {'retorno': 0.4152542372881356,
   'risco': 0.14878483916575333,
   'sharpe': 2.6565491450900027}),
 ('YDUQ3',
  {'retorno': 0.4042797095911348,
   'risco': 0.16469540900104315,
   'sharpe': 2.333275177019056}),
 ('JBSS3',
  {'retorno': 0.12839506172839513,
   'risco': 0.048888257989424776,
   'sharpe': 2.2172003296137595}),
 ('OIBR3',
  {'retorno': 0.2768361581920903,
   'risco': 0.12893469802305893,
   'sharpe': 1.991986347586258}),
 ('BRAP4',
  {'retorno': 0.25726392251815977,
   'risco': 0.11949040475233842,
   'sharpe': 1.9856315911718974}),
 ('MEAL3',
  {'retorno': 0.2391930835734869,
   'risco': 0.12013352872201775,
   'sharpe': 1.8245787492073708}),
 ('GOLL4',
  {'retorno': 0.3273159144893112,
   'risco': 0.17418522071319006,
   'sharpe': 1.764

Qual é a ação com o maior índice de Sharpe? (ou seja, com a maior relação retorno-risco)

In [74]:
desempenho_em_ordem[0]

('EEEL3',
 {'retorno': 0.16666666666666666,
  'risco': 0.019965281992164446,
  'sharpe': 7.346085405867411})

## Portfólios

Até agora, nós só falamos sobre ações separadamente. 
Na prática, você não precisa colocar todo o seu dinheiro em um único investimento.
Você pode ter um conjunto de investimentos e, assim, diversificar o seu risco.  
Assim, você não corre o risco do teu único investimento ir mal e você perder tudo.  
Um conjunto de investimentos se chama um **portfólio**.  

A grande questão ao se montar um portfólio é quanto dinheiro colocar em cada investimento.  

Vamos considerar um portfólio muito simples:  Duas ações. Bradesco (BBDC4) e Itaú (ITUB4).

Você aloca uma fração $f$ do seu patrimônio em Bradesco.  
O resto ($1-f$) você aloca em Itaú.  
Note que a soma dos dois dá 1, ou seja, 100% do seu patrimônio. 

Quanto você deve comprar de Bradesco?  
Em outras palavras, qual é o valor de $f$?  

Nós continuamos querendo muito retorno e pouco risco. Ou seja, continuamos querendo um índice de Sharpe alto. Mas, agora, não estamos interessados no índice de Sharpe do Bradesco, nem do Itaú: estamos interessados no índice de Sharpe do _conjunto_, ou seja, do _portfólio_.

Para calcularmos o índice de Sharpe do portfólio, precisamos saber o preço do portfólio.  
Pensa comigo:
Finge que Bradesco custa 100 e Itaú custa 90.
Eu tenho 50% em Bradesco e 50% em Itaú.  
Quando custa o meu portfólio?
$$
50\% \times 100 + 50\%\times 90 = 95
$$

Faz sentido?  

De forma genérica, se eu tenho $f$ em uma ação que vale $p_1$ e $(1-f)$ em uma ação que vale $p_2$, o preço do meu portfólio é

$$
p_{\text{portfolio}} = f\times p_1 + (1-f)\times p_2
$$

Podemos criar uma função para calcular isso:

In [79]:
def portfolio(f, p1, p2):
    return(f * p1 + (1-f)*p2)

Com isso, podemos calcular o índice de Sharpe do portfólio. A questão é quanto dinheiro colocar em cada ação para que o índice de Sharpe do portfólio seja o maior possível (isso é, para que o portfólio tenha a melhor relação retorno-risco).

É isso o que a função a seguir faz. 

In [109]:
def portfolio_otimo(ticker_1, ticker_2):
     
    import numpy as np #ignora isso
    
    p1 = busca_preco_fechamento(ticker_1)
    p2 = busca_preco_fechamento(ticker_2)
    
    best = {'f': 0, 'sharpe': 0}
    
    for f in np.arange(0.0,1.0,0.01): #o np.arange aqui é tipo o range que você já conhece
        p = portfolio(f=f, p1 = p1, p2 = p2)
        IS = Sharpe(p)
        if IS > best['sharpe']:
            best['f'] = f
            best['sharpe'] = IS
    
    return(best)

Vamos ver o que essa função diz sobre quanto colocar em Bradesco e quanto colocar em Itaú:

In [124]:
portfolio_otimo('BBDC4', 'ITUB4')

{'f': 0.45, 'sharpe': 0.8663345809282983}